# Analyze DTI - Preprocessing data without TOPUP (for participants for which it cannot be used)

In [1]:
# import packages -
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import os
import glob
import subprocess
import sys
import json
import platform
import socket

In [2]:
# parameters
# -----------------------------------------------------------------------------
MANUALLY_EXCLUDE_SUBJECTS = []

running_on = 'server' if 'Linux' in platform.system() else 'my_mac'

if running_on == 'my_mac':
    data_path = '/Users/ranigera/Dropbox/DTI_tests'
    preproc_path = '/Users/ranigera/Dropbox/DTI_tests/preproc'
    dti_path = '/Users/ranigera/Dropbox/DTI_tests/dti'
    stats_path = '/Users/ranigera/Dropbox/DTI_tests/stats'
    models_path = stats_path + '/models'
    launch_files_path = '/Users/ranigera/Dropbox/DTI_analysis/launch_files'
else:
    data_path = '/export2/DATA/HIS/HIS_server/BIDS'
    preproc_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc'
    dti_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/dti'
    stats_path = '/export2/DATA/HIS/HIS_server/analysis/dwi_data/stats'
    models_path = stats_path + '/models'
    launch_files_path = '/export2/DATA/HIS/HIS_server/codes_dwi/launch_files'

fmriPrepAnatomyDerivatives_path = data_path + '/derivatives/fmriprep'

expectedVolums = {
    'AP': 69,
    'PA' : 7,
    }
expectedB0s_indxs = {
    'AP_before': [0, 1, 18, 35, 52],
    'AP_after': [0, 1, 18, 35, 52]
    }

n_cores_TOPUP = 2

# setting EDDY stuff:
EDDY_command = 'eddy_openmp' if running_on == 'server' else 'eddy' # for the boost server
EDDY_command = 'eddy_cuda10.2' if running_on == 'server' else 'eddy' # for the cheshire server
ssh_command_for_cheshire_server = 'ssh shirangera@cheshire.tau.ac.il' if 'boost' in socket.gethostname() else ''

n_expected_EDDY_output_files = 13
n_cores_EDDY = 4 # relevant only for running using files (currently disabled as I run it on cheshire's GPU)

n_expected_DTIFIT_output_files = 10

In [3]:
# Define functions
# -----------------------------------------------------------------------------
def createSubjectScansBaseNames(subjFolder, data_path):
    sub = int(subjFolder.split("-",1)[1])
    group = '1day' if sub < 200 else '3day'
    last_sess = group[0]
    DWI_path_before = os.path.join(data_path, subjFolder, 'ses-1/dwi/')
    DWI_path_after = os.path.join(data_path, subjFolder, f'ses-{last_sess}/dwi/')
    scansBaseNames = {
        'AP_before': f'{os.path.join(DWI_path_before, "sub-" + str(sub) + "_ses-1_acq-ap_run-01_dwi")}',
        'AP_after' : f'{os.path.join(DWI_path_after, "sub-" + str(sub) + "_ses-" + last_sess + "_acq-ap_run-02_dwi")}'
        }
    return scansBaseNames

def get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs):
    sub_B0s_files = []
    for scan in scansBaseNames.keys():
        for B0ind in B0s_indxs[scan]:
            B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
            sub_B0s_files.append(B0_file_name)
    return sub_B0s_files


In [4]:
# Get folders and remove excluded subjects
# -----------------------------------------------------------------------------
print('>> Get sub folders')
subjFolders = [el for el in os.listdir(data_path) if 'sub' in el]

if running_on == 'my_mac':
    print('>> Get exclusion list')
    with open('/Users/ranigera/Google_Drive_TAU/Experiments/HIS_STUDY/Analysis/codes/paths_and_vars.py') as txtFile:
        txt = txtFile.read()
    participantsToExclude = [int(el) for el in txt.split('participantsToExclude = [')[1].split(']')[0].replace('\n','').replace('\n','').replace("'","").split(',')]

    print('>> Remove sub folders of excluded participants in case they are there')
    subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in participantsToExclude]

if MANUALLY_EXCLUDE_SUBJECTS:
    subjFolders = [el for el in subjFolders if int(el.split('-')[1]) not in MANUALLY_EXCLUDE_SUBJECTS]
    
subjFolders.sort()

>> Get sub folders


## Check for missing scans or wrong phase encoding directions for ALL SUBJECTS 

In [5]:
print ('>> Verify that all the scans exist and that the phase encoding directions are as they should.')
subjectsWithAProblem = []
for subjFolder in subjFolders:
    sub = int(subjFolder.split("-",1)[1])
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    for scan in scansBaseNames.keys():
        # print(scansBaseNames[scan] + '.json')
        # print(scanData['PhaseEncodingDirection'])
        if not os.path.exists(scansBaseNames[scan] + '.json'):
            subjectsWithAProblem.append(sub)
            print(' *** Scan not found: ' + scansBaseNames[scan] + '.json')
            continue
        with open(scansBaseNames[scan] + '.json') as json_file:        
            scanData = json.load(json_file)
            if ('acq-ap_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j-') or \
                ('acq-pa_' in scansBaseNames[scan] and scanData['PhaseEncodingDirection'] != 'j'):
                subjectsWithAProblem.append(sub)
                print(' *** There is a problem with the scanning directions: ' + scansBaseNames[scan] + '.json is defined as ' + scanData['PhaseEncodingDirection'] + '.')
                continue

subjectsWithAProblem = list(set(subjectsWithAProblem))
subjectsWithAProblem.sort()

>> Verify that all the scans exist and that the phase encoding directions are as they should.
 *** Scan not found: /export2/DATA/HIS/HIS_server/BIDS/sub-259/ses-3/dwi/sub-259_ses-3_acq-ap_run-02_dwi.json


In [7]:
# Based on the other file:
UnanalyzableSubjects = [211, 259]
#subjToAnalyzeWithOnlyAP = [101, 110, 204, 207, 209, 255]
subjToAnalyzeWithOnlyAP = [110, 204, 207, 209, 255]

In [8]:
# Take only subjects that can be analyzed but without opposite phase encoding (PA) scans
# -----------------------------------------------------------------------------
print('>> SELECT SUBJECTS WITH TWO SCANS BUT WITHOUT OPPOSITE PHASE ENCODING')
subjFolders =[el for el in subjFolders if int(el.split('-')[1])  in subjToAnalyzeWithOnlyAP]
subjFolders

>> SELECT SUBJECTS WITH TWO SCANS BUT WITHOUT OPPOSITE PHASE ENCODING


['sub-101', 'sub-110', 'sub-204', 'sub-207', 'sub-209', 'sub-255']

## Get b0 volume indices and perform bval & bvec QA

In [9]:

for subjFolder in subjFolders:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    print('> Verify that data points in the bval files is as expected.')
    if pd.read_csv(scansBaseNames['AP_before'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['AP_after'] + '.bval', header=None, sep=' ').T.shape[0] != expectedVolums['AP']:
            print(f' *** The number of data points in the bval for one of the scans for subjetc {subjFolder} is not as expected.')
            raise Exception(f'The number of data points in the bval for one of the scans for subjetc {subjFolder} is not as expected.')

    print('> Verify that data points in the bvec files is as expected.')
    if pd.read_csv(scansBaseNames['AP_before'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP'] or \
        pd.read_csv(scansBaseNames['AP_after'] + '.bvec', header=None, sep=' ').T.shape[0] != expectedVolums['AP']:
            print(f' *** The number of data points in the bvec for one of the scans for subjetc {subjFolder} is not as expected.')
            raise ValueError(f'The number of data points in the bvec for one of the scans for subjetc ' + str(subjFolder) + ' is not as expected.')

    print('> Extract B0s:')
    B0s_indxs = {}
    for scan in scansBaseNames.keys():
        B0s=pd.read_csv(scansBaseNames[scan] + '.bval', header=None, sep=' ').T
        B0s.columns = ['bval']
        B0s_indxs[scan] = list(B0s[B0s.bval < 20].index)

    print('> Verify that b0 quantity and indices are as expected.')
    if B0s_indxs != expectedB0s_indxs:
        print(f' *** The indices of the b0s for one of the scans for subjetc {subjFolder} are not as expected.')
        raise ValueError(f'The indices of the b0s for one of the scans for subjetc ' + str(subjFolder) + ' are not as expected.')


> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as expected.
> Verify that data points in the bvec files is as expected.
> Extract B0s:
> Verify that b0 quantity and indices are as expected.
> Verify that data points in the bval files is as 

## Preprocessing

#### Create pre-processing folder

In [10]:
print('> Create preprocessing folders')
for subjFolder in subjFolders:
    try:
        os.makedirs(os.path.join(preproc_path, subjFolder), exist_ok=False)
        print('>> Created folder: ' + os.path.join(preproc_path, subjFolder))
    except:
        pass

> Create preprocessing folders


### (1) Preperations of B0s etc (The parllel stuff to run like when preparing before TOPUP)


#### Extract B0 volumes

In [11]:
print('> Extract B0s (using the fslroi):')
for subjFolder in subjFolders:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    for scan in scansBaseNames.keys():
        for B0ind in B0s_indxs[scan]:
            B0_file_name = os.path.join(preproc_path, subjFolder, subjFolder + '_' + scan + "_b0_volInd-" + str(B0ind) + ".nii.gz")
            if not os.path.isfile(f"{B0_file_name}"):
                print(f'>> runs: fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
                os.system(f'fslroi {scansBaseNames[scan]}.nii.gz {B0_file_name} {B0ind} 1')
print(f">> COMPLETED.")

> Extract B0s (using the fslroi):
>> COMPLETED.


#### Merge B0 volumes for TOPUP

In [12]:
print('> Merge B0s for each session (using the fslmerge):')
#print([scan for scan in sub_B0s_files if 'AP_before' in scan])
#print([scan for scan in sub_B0s_files if 'PA_before' in scan])
for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    sub_B0s_files = get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs)
    # merge the B0s for the current subject for each time (before/after):
    for time in ['before', 'after']:
        output_base_name = f'{os.path.join(preproc_path, subjFolder,subjFolder)}_AP_{time}_b0s'
        if not os.path.isfile(f"{output_base_name}.nii.gz"):
            print(f'>> runs: fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]))
            os.system(f'fslmerge -t {output_base_name}' + ' ' + ' '.join([scan for scan in sub_B0s_files if f'AP_{time}' in scan]))
print(f">> COMPLETED.")

> Merge B0s for each session (using the fslmerge):
>> COMPLETED.


#### Generate the acqparams.txt files

In [14]:
print('> Create the acqparams.txt files (one per session [''before'' and ''after''])')

for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)
    sub_B0s_files = get_sub_B0_files(subjFolder, scansBaseNames, B0s_indxs)

    # first get the totalReadoutTime from the json files
    total_readout_time = {}
    for scanBaseName in scansBaseNames.keys():
        with open(scansBaseNames[scanBaseName] + '.json') as json_file:        
            scanData = json.load(json_file)
            total_readout_time[scanBaseName] = scanData['TotalReadoutTime']

    for time in ['before', 'after']:
        acqPars=[f'0 -1 0 {total_readout_time[f"AP_{time}"]}' for scan in sub_B0s_files if f'AP_{time}' in scan] + [f'0 1 0 {total_readout_time[f"PA_{time}"]}' for scan in sub_B0s_files if f'PA_{time}' in scan]
        # write a list of strings to a file (one string per line):
        with open(os.path.join(preproc_path, subjFolder, subjFolder + f'_{time}_acqparams.txt'), 'w') as f:
            for item in acqPars:
                f.write("%s\n" % item)
print(f">> COMPLETED.")

> Create the acqparams.txt files (one per session [before and after])
>> COMPLETED.


### (2) EDDY
* A tool for correcting Eddy currents (and motion)

#### Average the B0 volums (instead of the TOPUP the .iout file)

In [15]:
print(f">> Average the B0 volums (instead of the TOPUP .iout file). - runs seperately for each time [''before'' and ''after'']")
for subjFolder in subjFolders:
      for time in ['before', 'after']:
            output_file_name = f"{os.path.join(preproc_path, subjFolder,subjFolder)}_AP_{time}_b0s_iout_avg.nii.gz"
            if not os.path.isfile(f"{output_file_name}"):
                  print(f">> runs: fslmaths {os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s -Tmean {output_file_name}")
                  os.system(f"fslmaths {os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s -Tmean {output_file_name}")
print(f">> COMPLETED.")

>> Average the B0 volums (instead of the TOPUP .iout file). - runs seperately for each time [''before'' and ''after'']
>> COMPLETED.


#### Run BET on the averaged B0s volume

In [16]:
print(f">> Extarct the brain form the averaged B0s volume - runs seperately for each time [''before'' and ''after'']")
for subjFolder in subjFolders:
      for time in ['before', 'after']:
            output_base_name = f"{os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s_iout_avg_brain"
            if not os.path.isfile(f"{output_base_name}.nii.gz") or not os.path.isfile(f"{output_base_name}_mask.nii.gz"):
                  print(f">> runs: bet {os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
                  os.system(f"bet {os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s_iout_avg {output_base_name} -m -f 0.2")
print(f">> COMPLETED.")

>> Extarct the brain form the averaged B0s volume - runs seperately for each time [''before'' and ''after'']
>> COMPLETED.


#### Create index.txt file
This file maps the volumes in the main DTI data to the relevant line in the acqparams.txt and in the movpar.txt (assessed movement parameters from TOPUP) files
* details in: https://www.youtube.com/watch?v=1T1cRnX7MpA

In [17]:
print(f">> create an index.txt file for each subject for each time [before and after] for the EDDY")
for subjFolder in subjFolders:
    for time in ['before', 'after']:
        ind_to_write = 1
        with open(os.path.join(preproc_path, subjFolder, f'{time}_index.txt'), 'w') as f:
            for i in range(expectedVolums['AP']):   
                if i > 0 and i in expectedB0s_indxs[f'AP_{time}']:
                    ind_to_write += 1
                f.write("%s\n" %ind_to_write)
print('>> COMPLETED.')


>> create an index.txt file for each subject for each time [before and after] for the EDDY
>> COMPLETED.


#### Run EDDY
* Correct for eddy currents and subject movement (and taking to account the suceptibility field calculated by TOPUP)

In [19]:
print('> Run EDDY using cheshire server GPU (one per subject per time [''before'' and ''after''])')
# files_to_run_each_time = 200
# counter=1
for subjFolder in subjFolders:
    # if int(subjFolder.split('-')[1])<265:
    #     continue

    # # Stop executing more:
    # if counter > files_to_run_each_time:
    #     break

    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    # check if all expected output files exist:
    for time in ['before', 'after']:
        
        eddy_output_files = glob.glob(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}*')) # get files in a directory
        expected_EDDY_out_files_exist = len(list(set(eddy_output_files))) >= n_expected_EDDY_output_files # check that the number of unique EDDY files is as expected

        # Run the launch file if not all output files present:
        if not expected_EDDY_out_files_exist:
            eddy_command_to_run = f"{ssh_command_for_cheshire_server} {EDDY_command} --imain={scansBaseNames[f'AP_{time}']}.nii.gz \\\n\
                --mask={os.path.join(preproc_path, subjFolder, subjFolder)}_AP_{time}_b0s_iout_avg_brain_mask \\\n\
                --index={os.path.join(preproc_path, subjFolder, time + '_index.txt')}\\\n\
                --acqp={os.path.join(preproc_path, subjFolder, subjFolder + '_' + time + '_acqparams.txt')} \\\n\
                --bvecs={scansBaseNames[f'AP_{time}']}.bvec \\\n\
                --bvals={scansBaseNames[f'AP_{time}']}.bval \\\n\
                --out={os.path.join(preproc_path, subjFolder,'eddy_unwarped_images_' + subjFolder)}_{time} \\\n\
                --verbose\
            "
            print(f'">> Running this command on cheshire:\n{eddy_command_to_run}\n')
            os.system(eddy_command_to_run)
            # counter+=1

print(f">> COMPLETED.")

> Run EDDY using cheshire server GPU (one per subject per time [before and after])
">> Running this command on cheshire:
ssh shirangera@cheshire.tau.ac.il eddy_cuda10.2 --imain=/export2/DATA/HIS/HIS_server/BIDS/sub-204/ses-1/dwi/sub-204_ses-1_acq-ap_run-01_dwi.nii.gz \
                --mask=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-204/sub-204_AP_before_b0s_iout_avg_brain_mask \
                --index=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-204/before_index.txt\
                --acqp=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-204/sub-204_before_acqparams.txt \
                --bvecs=/export2/DATA/HIS/HIS_server/BIDS/sub-204/ses-1/dwi/sub-204_ses-1_acq-ap_run-01_dwi.bvec \
                --bvals=/export2/DATA/HIS/HIS_server/BIDS/sub-204/ses-1/dwi/sub-204_ses-1_acq-ap_run-01_dwi.bval \
                --out=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-204/eddy_unwarped_images_sub-204_before \
                --verbose

#### Run EDDY QC

In [43]:
for subjFolder in subjFolders:
    # get the sub_B0s_files for the current subject:
    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    for time in ['before', 'after']:
        if not os.path.isdir(os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}.qc')):
            print(f"eddy_quad {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                -idx {os.path.join(preproc_path, subjFolder, f'{time}_index.txt')} \\\n\
                -par {os.path.join(preproc_path, subjFolder, subjFolder  + f'_{time}_acqparams.txt')} \\\n\
                -m   {os.path.join(preproc_path, subjFolder, f'{subjFolder}_AP_{time}_b0s_iout_avg_brain_mask.nii.gz')} \\\n\
                -b   {scansBaseNames[f'AP_{time}'] + '.bval'}\
                ")
            os.system(f"eddy_quad {os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                -idx {os.path.join(preproc_path, subjFolder, f'{time}_index.txt')} \\\n\
                -par {os.path.join(preproc_path, subjFolder, subjFolder  + f'_{time}_acqparams.txt')} \\\n\
                -m   {os.path.join(preproc_path, subjFolder, f'{subjFolder}_AP_{time}_b0s_iout_avg_brain_mask.nii.gz')} \\\n\
                -b   {scansBaseNames[f'AP_{time}'] + '.bval'}\
                ")




## DTI modelling

#### Create the DTI folders

In [22]:
print('> Create preprocessing folders')
for subjFolder in subjFolders:
    try:
        os.makedirs(os.path.join(dti_path, subjFolder), exist_ok=False)
        print('>> Created folder: ' + os.path.join(dti_path, subjFolder))
    except:
        pass

> Create preprocessing folders


In [29]:
print('> Run DTIFIT (one per subject per time [''before'' and ''after''])')
# files_to_run_each_time = 200
# counter=1
for subjFolder in subjFolders:

    scansBaseNames = createSubjectScansBaseNames(subjFolder, data_path)

    # check if all expected output files exist:
    for time in ['before', 'after']:
        
        dtifit_output_files = glob.glob(os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}*')) # get files in a directory
        expected_DTIFIT_out_files_exist = len(list(set(dtifit_output_files))) >= n_expected_DTIFIT_output_files # check that the number of unique EDDY files is as expected

        # Run the launch file if not all output files present:
        if not expected_DTIFIT_out_files_exist:
            dtifit_command = f"dtifit --data={os.path.join(preproc_path, subjFolder, f'eddy_unwarped_images_{subjFolder}_{time}')} \\\n\
                        --mask={os.path.join(preproc_path, subjFolder, f'{subjFolder}_AP_{time}_b0s_iout_avg_brain_mask')} \\\n\
                        --bvecs={scansBaseNames[f'AP_{time}'] + '.bvec'} \\\n\
                        --bvals={scansBaseNames[f'AP_{time}'] + '.bval'} \\\n\
                        --out={os.path.join(dti_path, subjFolder, f'dti_{subjFolder}_{time}')}"

            print(dtifit_command)
            os.system(dtifit_command)



> Run DTIFIT (one per subject per time [before and after])
dtifit --data=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-110/eddy_unwarped_images_sub-110_before \
                        --mask=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-110/sub-110_AP_before_b0s_iout_avg_brain_mask \
                        --bvecs=/export2/DATA/HIS/HIS_server/BIDS/sub-110/ses-1/dwi/sub-110_ses-1_acq-ap_run-01_dwi.bvec \
                        --bvals=/export2/DATA/HIS/HIS_server/BIDS/sub-110/ses-1/dwi/sub-110_ses-1_acq-ap_run-01_dwi.bval \
                        --out=/export2/DATA/HIS/HIS_server/analysis/dwi_data/dti/sub-110/dti_sub-110_before
dtifit --data=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-110/eddy_unwarped_images_sub-110_after \
                        --mask=/export2/DATA/HIS/HIS_server/analysis/dwi_data/preproc/sub-110/sub-110_AP_after_b0s_iout_avg_brain_mask \
                        --bvecs=/export2/DATA/HIS/HIS_server/BIDS/sub-110/ses-1/dwi